# Workflow for the CRC1333 project B07 - Technical Chemistry

**Import necessary packages.**

In [11]:
# import matplotlib.pyplot as plt
# import numpy as np
# import networkx as nx
# import pandas as pd

from datetime import datetime
from sdRDM import DataModel
from modules import GCParser
from modules import GstaticParser
from modules import MFMParser
from modules import Calculator
from modules import get_mfr
from modules import get_initial_time_current
from modules import assign_peaks
from pathlib import Path
# from DEXPI2sdRDM import DEXPI2sdRDM

**Load data model from git or markdown file**

In [12]:
lib = DataModel.from_markdown('specifications/datamodel_b07_tc.md')
# lib = DataModel.from_git(url='https://github.com/FAIRChemistry/datamodel_b07_tc.git')

**Set path to current working directory and available datasets.**

In [13]:
# Get the current working directory
current_directory = Path.cwd()


path_to_dataset = current_directory / 'datasets'

In [19]:
# List all files in the directory
files = path_to_dataset.iterdir()

# Filter the files with .json extension
json_files = {index:file for index, file in enumerate(files) if file.suffix == ".json"}

# Print the JSON file names

for index, file in json_files.items():
    print(f"{index}: {file.name}")

0: b07.json


**Choose dataset to parse.**

In [20]:
index_dataset = 0
DataModel.parse(json_files[index_dataset])


KeyError: 'repo'

**Load dataset for project B07**

In [ ]:
b07=DataModel.parse('./dataset_b07.json')

**Instantiate an experiment object which holds all the information about one single experiment.**

In [ ]:
experiment = lib.Experiment()

### **Parsing experimental data and metadata from various output files.**

### gstatic

**Create an instance of the ``GstaticParser`` to parse Gamry output files and show available files in the selected directory.**

In [ ]:
gstaticparser = GstaticParser(path_to_dataset / 'data')
dict_of_gstatic_files = gstaticparser.enumerate_available_files()
for index, gstatic_file in dict_of_gstatic_files.items():
    print(f"{index}: {gstatic_file}")
# available_files = gstaticparser.available_files
# print(available_files)

**Chose specific file.**

In [ ]:
gstatic_file = dict_of_gstatic_files[0]
gstatic_file

**Extract the metadata from it using the ``GstaticParser`` and load into the data model.**

In [ ]:
gstatic_metadata_df, gstatic_metadata = gstaticparser.extract_metadata(gstatic_file)
pot = lib.Measurement(measurement_type=lib.enums.MeasurementType.POTENTIOSTATIC, metadata=gstatic_metadata)
experiment.measurements = [pot]
b07.experiments=[experiment]
gstatic_metadata_df

### mfm

**Create an instance of the ``MFMParser`` to parse MFM output files and show available files in the selected directory.**

In [ ]:
mfmparser = MFMParser(path_to_dataset / 'data')
dict_of_mfm_files = mfmparser.enumerate_available_files()
for index, mfmfile in dict_of_mfm_files.items():
    print(f"{index}: {mfmfile}")

**Chose specific file**

In [ ]:
mfm_file = dict_of_mfm_files[3]
mfm_file

**Extract the experimental data from it using the ``MFMParser`` and load into the data model.**

In [ ]:
mfm_exp_data_df, mfm_exp_data = mfmparser.extract_exp_data(mfm_file)
mfm = lib.Measurement(
            measurement_type=lib.enums.MeasurementType.MFM.value,
            experimental_data=[value for value in mfm_exp_data.values()],
        )
b07.experiments[0].measurements.append(mfm)
mfm_exp_data_df

### gc

**Create an instance of the ``GCParser`` to parse GC output files and show available files in the selected directory.**

In [ ]:
gcparser = GCParser(path_to_dataset / 'data')
dict_of_gc_files = gcparser.enumerate_available_files()
for index, gc_file in dict_of_gc_files.items():
    print(f"{index}: {gc_file}")

**Chose file that contains the metadata**

In [ ]:
gcmd_file = dict_of_gc_files[0]

**Chose file that contains the experimental data**

In [ ]:
gced_file = dict_of_gc_files[1]

**Extract the metadata and experimental data from them using the ``GCParser`` and load into the data model.**

In [ ]:
gc_metadata_df, gc_metadata= gcparser.extract_metadata(gcmd_file)
gc_exp_data_df, gc_exp_data = gcparser.extract_exp_data(gced_file)
gc = lib.Measurement(
    measurement_type=lib.enums.MeasurementType.GC.value,
    metadata=[value for value in gc_metadata.values()],
    experimental_data=[value for value in gc_exp_data.values()]
)
b07.experiments[0].measurements.append(gc)
gc_metadata_df

In [ ]:
gc_exp_data_df

In [ ]:
type(gc_exp_data_df["Peak_number"].values.tolist()[0])

**Print current state of the data model**

In [ ]:
print(b07.json())

**Get initial current in mA and initial time in s.**

In [ ]:
I_init, t_init = get_initial_time_current(b07)
I_init, t_init

**Get mean flow rate at the time of the gc measurement.**

In [ ]:
inj_date, inj_dates, mfr, vol_flow_mean = get_mfr(b07, 10)
vol_flow_mean

**Assign peak areas to species.** \
The peak areas recorded by the GC have to be matched with the correct species. The individial ``Area`` is selected by its corresponding ``Peak_Number``. It is possible that the same species is accountable for multiple peaks, i.d. multiple peaks are assigned to the same species.


In [ ]:
gc_exp_data_df

In [ ]:
assign_peak_dict={
    'CO2': [2],
    'CO': [6],
    'H2': [1],
    'CH4': [3],
    'C2H4': [5],
    # 'C2H6': [4],
}
peak_area_dict = assign_peaks(b07, assign_peak_dict)

**Set calibration input values and import into the data model.** \
To determine the concentrations of the individual species, a calibration has to be performed in advance to match the individual values for ``Area`` with their corresponding concentrations.

In [ ]:
cali_input_dict={
    'CO2': [[0,38653],[0,50], lib.enums.Species.CARBONDIOXIDE],
    'CO':[[797,1328,7223],[0.5,1,5], lib.enums.Species.CARBONMONOXIDE],
    'H2':[[71,153,330], [5,10,20], lib.enums.Species.HYDROGEN],
    'CH4':[[5727,11991], [5,10], lib.enums.Species.METHANE],
    'C2H4':[[1122,4864,7297], [0.5,2,3], lib.enums.Species.ETHENE],
    'C2H6':[[0,12168], [0,5], lib.enums.Species.ETHANE],
}

In [ ]:
calibration_list = []
for value in cali_input_dict.values():
    calibration_list.append(
        lib.Calibration(
            peak_area=[
                lib.Data(  
                    values=value[0],
                    unit=lib.enums.Unit.VOLFRACTION
                )
            ],
            concentration=[
                lib.Data(  
                    values=value[1],
                    unit=lib.enums.Unit.VOLFRACTION
                )
            ],
            species=value[2]
        )
    )


In [ ]:
calc = lib.Calculation()
calc.calibrations = calibration_list
b07.experiments[0].calculations = calc

In [ ]:
print(b07.json())

Instantiate the ``Calculator`` module.

In [ ]:
calculator=Calculator(path_to_dataset=path_to_dataset)

Calibrate using the ``calibrate`` method of the ``Calculator`` module.

In [ ]:
cali_result_df=calculator.calibrate(b07)
cali_result_df

Calculate volume fractions using the ``calculate_volume_fractions`` method of the ``Calculator`` module.

In [ ]:
vol_frac_df = calculator.calculate_volume_fractions(peak_area_dict=peak_area_dict, calibration_result_df=cali_result_df)
vol_frac_df

Calculate the conversion factors of the individual species using the ``calculate_conversion_factors`` method of the ``Calculator`` module and the correction factors.

In [ ]:
f_corr_dict= {
    'H2':1.01,
    'CO':0.74,
    'CO2':1.00,
    'CH4':0.76,
    # 'C2H4':,
    # 'C2H6':,
}

In [ ]:
vol_frac_df.loc['H2'][0]

In [ ]:
f_conv = calculator.calculate_conversion_factor(volume_fractions=vol_frac_df, correction_factors=f_corr_dict)
f_conv

In [ ]:
vol_flow_r_df = calculator.calculate_vol_flow_r(conversion_factor = f_conv, volumetric_flow_measured = mfm_exp_data_df )
vol_flow_r_df

In [ ]:
# class DirectedGraph:
    
#     def __init__(self, name, connections):
#         self.name = name
#         self.connections = connections
#         self.connec_tuple = []
#         self.make_connectivity()

        
#     def make_connectivity(self):
#         if len(self.connections) == 0: 
#             return([])
#         for connection in self.connections:
#             self.connec_tuple.append((self.name, connection))

In [ ]:
# mod_1 = DirectedGraph(input0, [tubing0])
# mod_2 = DirectedGraph(input1, [tubing1])
# mod_3 = DirectedGraph(tubing0, [valve0])
# mod_4 = DirectedGraph(tubing1, [valve0])
# mod_5 = DirectedGraph(valve0, [tubing2])
# mod_6 = DirectedGraph(fc, [tubing2])
# mod_7 = DirectedGraph(tubing2, [valve1])
# mod_8 = DirectedGraph(valve1, [tubing4])
# mod_9 = DirectedGraph(tubing4, [valve2])
# mod_10 = DirectedGraph(valve2, [tubing5])
# mod_11 = DirectedGraph(tubing5, [gc])
# mod_12 = DirectedGraph(tubing5, [fr])

# modules = [mod_1, mod_2, mod_3, mod_4, mod_5, mod_6, mod_7, mod_8, mod_9, mod_10, mod_11, mod_12]
# module_names = [flow_mod.name for flow_mod in modules]

In [ ]:
# area = exp_df['Area']
# peak_area_dict = {
#     'co2': area.iloc[[1]].sum(),
#     'co': area.iloc[[5,6]].sum(),
#     'h2': area.iloc[[0]].sum(),
#     'ch4': area.iloc[[2,3]].sum(),
#     'c2h4': area.iloc[[4]].sum(),
#     # 'c2h6': area.iloc[[0,1]].sum()
# }
# peak_area_dict

In [ ]:
# flowG = nx.DiGraph()
# flowG.add_nodes_from(module_names)
# for module in modules:
#     flowG.add_edges_from(module.connec_tuple)
# my_pos = nx.spring_layout(flowG, seed = 5) # this fixes the style of the graph, if it's ugly change the seed
# nx.draw(flowG, pos = my_pos, with_labels=True)